In [ ]:
#pip install selenium

In [ ]:
driver = webdriver.Chrome(executable_path="C:/Users/Owner/Downloads/chromedriver.exe", options=option)

In [ ]:
driver.get('https://www.niche.com/places-to-live/search/best-places-to-live/s/texas/')

In [ ]:
time.sleep(2)
search_textinput=driver.find_element(By.XPATH, "//input[@placeholder = 'Texas']")
search_textinput.send_keys('Dallas-Fort Worth Area')
search_textinput.send_keys(Keys.RETURN)

In [ ]:
#Identifying Suburbs Checkbox
checkbox_input=driver.find_element(By.XPATH,"//*[@id='maincontent']/div/div/div[1]/aside/div[3]/div/div/div[3]/label")

In [ ]:
#Clicking on Suburbs checkbox to activate it and narrow down number of neighborhoods results to scrape
time.sleep(2)
checkbox_input.click()

In [ ]:
#Checking current url
print(driver.current_url)

In [ ]:
#Identify Number of Pages by looking at tag of dropdown options at the bottom of the page
time.sleep(2)
no_pages_detect= driver.find_elements_by_tag_name('option')

In [ ]:
len(no_pages_detect)

In [ ]:
dfw_dict={'neighborhood':[],'neighborhood_description':[],'niche_grade':[],'population':[],'median_home_value':[],'median_rent':[],'median_household_income':[],'niche_page':[]
          ,'public_school_grade_list':[], 'crime_grade_list':[], 'housing_grade_list':[],'nightlife_grade':[]
          ,'good_for_families_grade':[], 'diversity_grade':[],'jobs_grade':[] , 'weather_grade':[],'cost_of_living_grade':[]
          , 'health_grade':[],'outdoor_activities_grade':[],'commute_grade':[]}

for page in range(1,len(no_pages_detect)+1): #Ideally this should be 'for page in range (1, len(no_pages_detect))'. However, due to recaptcha limitidation, we are breaking down the extrations into batches by defining the range manually. Make sure the driver is on the page# that is listed as the beginning of the range.
    if page < len(no_pages_detect): # The loops below are applicable to all but the last page of results as we do not know how many results are posted in the last page.
        for i in range (1,30):
            if i in [7,8,20,21]: #xpath numbers are not continuous. We are ignoring these xpath numbers as they are absent in the website. Inclduing them will result in errors
                continue
            
            city_page=driver.find_element_by_xpath("//*[@id='maincontent']/div/div/section/div[2]/ol/li[{}]/div/div/a".format(i))
            
            #scrolling to city prior to selecting
            driver.execute_script("arguments[0].scrollIntoView();", city_page)
            
            time.sleep(.2) #wating .2 second post scrolling to click
            
            #Click on Neighborhood name to navigate to Neighborhood Details Page
            driver.execute_script("arguments[0].click();", city_page)
            
            #Prociding 1.5 seconds for page to Load
            time.sleep(1.5)
            
            #Determining neighborhood for which we will be scraping data
            neighborhood=driver.find_element_by_xpath("//*[@id='header']/div/div[2]/div[1]/h1")
            neighborhood_name_main=(neighborhood.text)

            print("Entering {}'s Detail's Page".format(neighborhood_name_main))


            #Clicking on 'View Full Report' option to expand report prior to extraction. All data points will not be scraped if this report is not explanded
            expand_report=driver.find_element_by_xpath('//*[@id="report-card"]/div/div[2]/a')
            
            time.sleep(.1) # waiting for page to load
            
            #scrolling to expand report prior to clicking
            driver.execute_script("arguments[0].scrollIntoView();", expand_report)
            
            time.sleep(.05)# wait before clicing
            
            driver.execute_script("arguments[0].click();", expand_report)

            #Extracting Neighborhood Details from details page:
            for i in list(dfw_dict.keys()):
                
                #We will now assign values to the dfw_dict dictionary based on position of the keys(columns) in the dictionary
                
                if list(dfw_dict.keys()).index(i)==0: #Neighboord is the first key in the dfw_dictionary
                    try:
                        neighborhood=driver.find_element_by_xpath("//*[@id='header']/div/div[2]/div[1]/h1")
                        dfw_dict[i].append(neighborhood.text)
                        #print("Neighborhood name extraction complete!") # unmute only while debudding in case of errors
                    except:
                        dfw_dict[i].append("NA")
                        
                        
                    
                elif list(dfw_dict.keys()).index(i)==1: 
                    try:
                        neighborhood_description=driver.find_element_by_xpath('//*[@id="header"]/div/div[2]/div[1]/ul[1]/li[2]')
                        dfw_dict[i].append(neighborhood_description.text)
                        #print("Neighborhood description extraction complete!") # unmute only while debudding in case of errors
                    except:
                        dfw_dict[i].append("NA")
                        
                elif list(dfw_dict.keys()).index(i)==2:
                    try:
                        niche_grade=driver.find_element_by_xpath('//*[@id="report-card"]/div/div[1]/div/div[1]/div/div/div/div[1]/div')
                        dfw_dict[i].append(niche_grade.text)
                    except:
                        dfw_dict[i].append("NA")
                        #print("Niche grade extraction complete!") # unmute only while debudding in case of errors
                    

                elif list(dfw_dict.keys()).index(i)==3:
                    try:
                        population=driver.find_element_by_xpath("//*[@id='about']/div[2]/div[1]/div/div/div[2]/div[2]/span")
                        #scrolling to population part of the page
                        driver.execute_script("arguments[0].scrollIntoView();", expand_report)
                        time.sleep(.2)
                        dfw_dict[i].append(population.text)
                        #print("Population extraction complete!") # unmute only while debudding in case of errors
                    except:
                        dfw_dict[i].append("0")


                elif list(dfw_dict.keys()).index(i) ==4:
                    try:
                        home_value=driver.find_element_by_xpath('//*[@id="real-estate"]/div[2]/div[1]/div/div/div[1]/div[2]/span')
                        dfw_dict[i].append(home_value.text)
                        #print("Median home value extraction complete!") # unmute only while debudding in case of errors
                    except:
                        dfw_dict[i].append("0")
                        #print("Median home value not available!") # unmute only while debudding in case of errors

                elif list(dfw_dict.keys()).index(i) ==5:
                    try:
                        median_rent=driver.find_element_by_xpath('//*[@id="real-estate"]/div[2]/div[1]/div/div/div[2]/div[2]/span')
                        dfw_dict[i].append(median_rent.text)
                        #print("Median rent extraction complete!") # unmute only while debudding in case of errors
                    except:
                        dfw_dict[i].append("0")
                        #print("Median rent not available!") # unmute only while debudding in case of errors
                        

                elif list(dfw_dict.keys()).index(i) ==6:
                    try:
                        median_household_income=driver.find_element_by_xpath('//*[@id="working-in"]/div[2]/div[2]/div/div/div[1]/div[2]/span')
                        dfw_dict[i].append(median_household_income.text)
                        #print("Median household income extraction complete!") # unmute only while debudding in case of errors
                    except:
                        dfw_dict[i].append("0")
                        #print("Median household income not available!") # unmute only while debudding in case of errors
                
                elif list(dfw_dict.keys()).index(i) ==7:
                    dfw_dict[i].append(page)
                    #print("Median household income extraction complete!") # unmute only while debudding in case of errors

                else:
                    try:
                        n=list(dfw_dict.keys()).index(i)-7
                        element=driver.find_element_by_xpath("//*[@id='report-card']/div/div[1]/div/div[2]/ol/li[{}]/div/div[2]".format(n))
                        dfw_dict[i].append(element.text)
                        n=n+1
                        #print("Other metrics extraction complete!") # unmute only while debudding in case of errors
                    except:
                        dfw_dict[i].append("NA")
                        #print("Other metrics not available!") # unmute only while debudding in case of errors

                    
            print("All data extraction complete")
            
            #scrolling back to top of page prior to clicking back
            driver.execute_script("window.scrollTo(0,0)")
            time.sleep(1)
            #navigating back to results page in order to select next city for scraping details
            driver.execute_script("window.history.go(-1)")
            print("Returning to Main Neighborhood Page")
            time.sleep(2) #holding a 2 second wait to allow results page ample time to load. Increase this time if code is executing faster than pages are loading
            
        #scrolling to bottom of the page prior to clicking next page button
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(.5) #waiting half a second prior to clicking next
        #At this stage, scraping of data in the current results page has been compelte.Code below will take us to the next page of results   
        next_page=driver.find_element(By.XPATH, "//li[@class = 'pagination__next']")
        driver.execute_script("arguments[0].click();", next_page)
        time.sleep(3) #holding a 3 second wait to allow next page of results ample time to load. Increase this time if code is executing faster than pages are loading

            
      
    
    
    #scraping results from last Page of results
    else:
        #Determining number of results in the last page of results
        no_neighbords_in_last_results_page=[]
        neighborhoods = driver.find_elements_by_css_selector('h2[class="search-result__title"]')
        
        for i in range (1,len(neighborhoods)+3):
            if i in [7,8,20,21]: #skipping these numbers as they do't exists in xpath
                continue
            city_page=driver.find_element_by_xpath("//*[@id='maincontent']/div/div/section/div[2]/ol/li[{}]/div/div/a".format(i))

            #Click on City name to go to Details Page
            driver.execute_script("arguments[0].click();", city_page)

            neighborhood=driver.find_element_by_xpath("//*[@id='header']/div/div[2]/div[1]/h1")
            neighborhood_name_main=(neighborhood.text)

            print("Entering {}'s Detail's Page".format(neighborhood_name_main))


            #Clicking on 'View Full Report' option to expand report prior to extraction
            expand_report=driver.find_element_by_xpath('//*[@id="report-card"]/div/div[2]/a')
            driver.execute_script("arguments[0].click();", expand_report)

            #Extracting Neighborhood Details from details page
            for i in list(dfw_dict.keys()):
                if list(dfw_dict.keys()).index(i)==0:
                    try:
                        neighborhood=driver.find_element_by_xpath("//*[@id='header']/div/div[2]/div[1]/h1")
                        dfw_dict[i].append(neighborhood.text)
                        #print("Neighborhood name extraction complete!")
                    except:
                        dfw_dict[i].append("NA")
                        
                        
                    
                elif list(dfw_dict.keys()).index(i)==1:
                    try:
                        neighborhood_description=driver.find_element_by_xpath('//*[@id="header"]/div/div[2]/div[1]/ul[1]/li[2]')
                        dfw_dict[i].append(neighborhood_description.text)
                        #print("Neighborhood name extraction complete!")
                    except:
                        dfw_dict[i].append("NA")
                        
                elif list(dfw_dict.keys()).index(i)==2:
                    try:
                        niche_grade=driver.find_element_by_xpath('//*[@id="report-card"]/div/div[1]/div/div[1]/div/div/div/div[1]/div')
                        dfw_dict[i].append(niche_grade.text)
                    except:
                        dfw_dict[i].append("NA")
                    

                elif list(dfw_dict.keys()).index(i)==3:
                    try:
                        population=driver.find_element_by_xpath("//*[@id='about']/div[2]/div[1]/div/div/div[2]/div[2]/span")
                        dfw_dict[i].append(population.text)
                        #print("Population extraction complete!")
                    except:
                        dfw_dict[i].append("0")


                elif list(dfw_dict.keys()).index(i) ==4:
                    try:
                        home_value=driver.find_element_by_xpath('//*[@id="real-estate"]/div[2]/div[1]/div/div/div[1]/div[2]/span')
                        dfw_dict[i].append(home_value.text)
                        #print("Median home value extraction complete!")
                    except:
                        dfw_dict[i].append("0")

                elif list(dfw_dict.keys()).index(i) ==5:
                    try:
                        median_rent=driver.find_element_by_xpath('//*[@id="real-estate"]/div[2]/div[1]/div/div/div[2]/div[2]/span')
                        dfw_dict[i].append(median_rent.text)
                    except:
                        dfw_dict[i].append("0")
                    #print("Median rent extraction complete!")

                elif list(dfw_dict.keys()).index(i) ==6:
                    try:
                        median_household_income=driver.find_element_by_xpath('//*[@id="working-in"]/div[2]/div[2]/div/div/div[1]/div[2]/span')
                        dfw_dict[i].append(median_household_income.text)
                        #print("Median household income extraction complete!")
                    except:
                        dfw_dict[i].append("0")
                
                elif list(dfw_dict.keys()).index(i) ==7:
                    dfw_dict[i].append(page)

                else:
                    try:
                        n=list(dfw_dict.keys()).index(i)-7
                        element=driver.find_element_by_xpath("//*[@id='report-card']/div/div[1]/div/div[2]/ol/li[{}]/div/div[2]".format(n))
                        dfw_dict[i].append(element.text)
                        n=n+1
                        #time.sleep(1) 
                    except:
                        dfw_dict[i].append("NA")
                        
            print("All data extraction complete")
            driver.execute_script("window.history.go(-1)")
            print("Returning to Main Neighborhood Page")
            time.sleep(3)
            
driver.quit()

In [ ]:
len(df)

In [ ]:
df.to_csv('dfw_data.csv')